In [1]:
from fpl_helpers import *
import pandas as pd

In [2]:
elements_df, teams_df = get_tables()
global PLAYER_IDS
PLAYER_IDS = elements_df[['full_name', 'id']].set_index('full_name').to_dict()['id']


In [3]:
#forwards = elements_df[(elements_df['minutes']>0) & (elements_df['element_type']==4)].sort_values(by='total_points', ascending=False)[PLAYER_COLS].head(20)
#forwards['difficulty'] = forwards['full_name'].apply(lambda x: np.mean(difficulty_range(PLAYER_IDS[x], gwrange=1)))
#forwards[:20].sort_values(by='difficulty')

In [ ]:
def get_team(team_id, gw):
  url = f"https://fantasy.premierleague.com/api/entry/{team_id}/event/{gw}/picks/"
  r = requests.get(url)
  json = r.json()
  return json

player_ids = [str(i['element']) for i in get_team(3053203, 4)['picks']]

In [ ]:
compare_players(['Bukayo Saka', 'Sven Botman'], metrics=ATTACKER_METRICS, agg='mean')

/Users/geirfreysson/Code/fpl/fpl/fpl_helpers.py:184: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  result = result.style.apply(highlight_max, axis=1).apply(highlight_min, axis=1).applymap(custom_format)


ValueError: Styles supplied as string must follow CSS rule formats, for example 'attr: val;'. '3.0, 2.0, 5.0, 3.0, 2.0, 4.0' was given.

In [ ]:
player_names = {v: k for k, v in PLAYER_IDS.items()}
compare_players([player_names[int(i)] for i in player_ids[1:4]] + ["Sven Botman"] , metrics=DEFENDER_METRICS, agg='mean')
compare_players([player_names[int(i)] for i in player_ids[4:9]], metrics=ATTACKER_METRICS, agg='mean')
compare_players([player_names[int(i)] for i in player_ids[9:13]] + ["Odsonne Édouard"], metrics=ATTACKER_METRICS, agg='mean')

,Manuel Akanji,Matty Cash,Pervis Estupiñán,Sven Botman
Difficulty,"[2.0, 2.0, 4.0, 3.0, 4.0, 2.0]","[3.0, 3.0, 2.0, 2.0, 2.0, 2.0]","[2.0, 3.0, 4.0, 5.0, 2.0, 2.0]","[2.0, 2.0, 3.0, 2.0, 2.0, 4.0]"
Total Points,1.333333,5.666667,1.000000,3.666667
Minutes,60.000000,90.000000,60.000000,58.666667
Clean Sheets,0.000000,0.000000,0.000000,0.333333
Goals Conceded,0.666667,1.666667,1.333333,0.333333
Own Goals,0.000000,0.333333,0.000000,0.000000
Yellow Cards,0.000000,0.333333,0.000000,0.000000
Red Cards,0.000000,0.000000,0.000000,0.000000
Bonus,0.000000,1.000000,0.000000,1.000000
Bps,11.333333,17.000000,9.333333,15.000000


,James Maddison,Moussa Diaby,Bukayo Saka,Mohamed Salah,Bryan Mbeumo
Difficulty,"[4.0, 4.0, 2.0, 2.0, 2.0, 3.0]","[3.0, 3.0, 2.0, 2.0, 2.0, 2.0]","[3.0, 2.0, 5.0, 3.0, 2.0, 4.0]","[2.0, 3.0, 3.0, 2.0, 2.0, 2.0]","[2.0, 2.0, 4.0, 2.0, 3.0, 2.0]"
Total Points,6.000000,6.333333,6.000000,7.666667,4.000000
Goals Scored,0.666667,0.333333,0.333333,0.333333,0.333333
Expected Goals,0.526667,0.126667,0.603333,0.406667,0.320000
Assists,0.000000,0.666667,0.666667,1.000000,0.000000
Expected Assists,0.276667,0.173333,0.486667,0.553333,0.223333
Expected Goal Involvements,0.803333,0.300000,1.090000,0.960000,0.543333
Bonus,0.666667,0.666667,0.333333,0.666667,0.333333
Bps,26.000000,22.666667,22.666667,26.666667,12.000000
Influence,40.866667,27.200000,36.666667,39.333333,19.000000


KeyError: 'Odsonne Édouard'

In [ ]:
def compare_players(player_names, metrics, agg=None, start_gameweek=None, gameweeks=3):
    def highlight_max(s):
        if type(s) == list:
            s = np.mean(s)
        is_max = s == s.max()
        return ['background-color: rgba(0,200,0,0.2)' if cell else '' for cell in is_max]
    def highlight_min(s):
        if type(s) == list:
            s = np.mean(s)
        is_min = s == s.min()
        return ['background-color: rgba(200,0,0,0.2)' if cell else '' for cell in is_min]

    columns = []
    for player_name in player_names:
        fixture_difficulty = difficulty_range(PLAYER_IDS[player_name])
        player_metrics = player_past_metrics(PLAYER_IDS[player_name], metrics, agg=agg, start_gameweek=start_gameweek, gameweeks=gameweeks)
        if type(list(player_metrics.values())[0]) == list:
            player_metrics_values = [", ".join([str(j) for j in i]) for i in list(player_metrics.values())]
        else:
            player_metrics_values = list(player_metrics.values())
        player_metrics_values.insert(0, fixture_difficulty)
        
        columns.append(pd.Series(player_metrics_values, name=player_name))
    metrics_list = metrics.copy()
    metrics_list.insert(0, 'Difficulty')
    df = pd.DataFrame(
        data = {
            'Metric': metrics_list
        }
    )
    df = pd.concat(columns, axis=1)
    df.index = [i.replace("_", " ").title() for i in metrics]
    result = df.round(1)
    result = result.style.apply(highlight_max, axis=1).apply(highlight_min, axis=1).format("{:.2f}")
    return result
compare_players(['Bukayo Saka', 'Raheem Sterling'], metrics=ATTACKER_METRICS, agg='mean')

ValueError: Length mismatch: Expected axis has 14 elements, new values have 13 elements